# Clever girl

In [ ]:
#!/usr/bin/env python

import gmpy2
from fractions import Fraction
from secret import p, q, s, X, Y
from flag import flag

assert gmpy2.is_prime(p) * gmpy2.is_prime(q) > 0
assert Fraction(p, p+1) + Fraction(q+1, q) == Fraction(2*s - X, s + Y)
print 'Fraction(p, p+1) + Fraction(q+1, q) = Fraction(2*s - %s, s + %s)' % (X, Y)

n = p * q
c = pow(int(flag.encode('hex'), 16), 0x20002, n)
print 'n =', n
print 'c =', c

The code is very short and simple: we need to break an RSA encryption, where $p,q$ satisfy some odd equation.

In particular we know that $$ \frac{p}{p+1}+\frac{q+1}{q} = \frac{2s-X}{s+Y} $$

We simplify the LHS in this way: $\frac{p}{p+1}+\frac{q+1}{q} = 2 + \frac1q-\frac1{p+1}$. Moving the $2$ on the other side, after some computations we get
$$ \frac{q-(p+1)}{(p+1)q} = \frac{X+2Y}{s+Y} $$
and after we multiply everything, we get the (not so) nice equation
$$ (q-p-1)(s+Y)=(X+2Y)(p+1)q $$

### Now what?

Now we start to make some assumptions or get some properties.

One of the first thing I tried was to factor $X+2Y$, since it's a number we know; unfortunately it was too big. 

Then we notice that $q>p+1$, and in particular $\gcd(q,p+1)=1$.

This is VERY useful, since we can do this trick: $q\mid (p+1)(s+Y)\implies q\mid s+Y$, and similarly $p+1\mid q(s+Y)\implies p+1\mid s+Y$. Given these poverful divisibilities, we write $$ q-p-1 \cdot \frac{s+Y}{q(p+1)} = X+2Y $$ where the fraction is actually an integer.

At this point I **really** wanted to factor $X+2Y$, but everything failed again (WolframAlpha, factordb.com, sage...).

### The lucky shot

So after some time, I said "what if that fraction is actually simply $1$?".

In this way, we'd have $q=p+1+X+2Y$, hence $n=pq=p^2+p(1+X+2Y)$, which is a quadratic equation in $p$ of which we know all coefficients!

I wrote some lines of code, and...

In [3]:
X = 153801856029563198525204130558738800846256680799373350925981555360388985602786501362501554433635610131437376183630577217917787342621398264625389914280509
Y = 8086061902465799210233863613232941060876437002894022994953293934963170056653232109405937694010696299303888742108631749969054117542816358078039478109426
n = 161010103536746712075112156042553283066813155993777943981946663919051986586388748662616958741697621238654724628406094469789970509959159343108847331259823125490271091357244742345403096394500947202321339572876147277506789731024810289354756781901338337411136794489136638411531539112369520980466458615878975406339

b = 1+(X+2*Y)
D = (b)^2+4*n
is_square(D)

True

Bingo! The discriminant is an integer!

So the quadratic equation has actually an integer solution, $p$. We can check that it is actually a prime number, but we know that already.

In [7]:
p =(-b+sqrt(D))/2
q = n/p

is_prime(Integer(p)) and is_prime(Integer(q))

True

And now it's the usual RSA decryption, with the added trick that $\gcd(\varphi(n),e)\neq1$.

Look at th roXen solution for details.

In [9]:
from Crypto.Util.number import *

phi = (p-1)*(q-1)
e = int("0x20002", 16)
d = inverse_mod(e//2, phi)

c = 64166146958225113130966383399465462600516627646827654061505253681784027524205938322376396685421354659091159523153346321216052274404398431369574383580893610370389016662302880230566394277969479472339696624461863666891731292801506958051383432113998695237733732222591191217365300789670291769876292466495287189494
m = power_mod(c, d, n)
long_to_bytes(m^(1/2))

'CCTF{4Ll___G1rL5___Are__T4len73E__:P}'